In [1]:
import os,sys, json
import magic, hashlib, os, traceback
import ntpath
from capstone import *
from capstone.x86 import *

import collections
import traceback
import lief
from elftools.elf.elffile import ELFFile

In [2]:
bin_path = '/home/raisul/DATA/temp/x86_O2_d4/' #'/home/raisul/DATA/x86_O2_d4_mingw32_PE' 

bin_files = [os.path.join(bin_path, f) for f in os.listdir(bin_path) ][0:10]

ground_truth_path ='/home/raisul/DATA/temp/ghidra_x86_O2_d4/' # '/home/raisul/ANALYSED_DATA/ghidra_x86_O2_d4_mingw32_PE'  

In [3]:
len(bin_files)

10

In [4]:

def get_ground_truth_ghidra(exe_path, text_section_offset , text_section_len):

    text_sextion_end = text_section_offset + text_section_len
    
    elf_file_name = os.path.basename(exe_path)
    ghidra_file_path = os.path.join(ground_truth_path, elf_file_name.split('.')[0]) + '.json'
    
    with open(ghidra_file_path, "r") as file:
        ghidra_data = json.load(file)

    ground_truth_offsets = list(ghidra_data.keys())

    ground_truth_offsets = [int(i) for i in ground_truth_offsets]
    ground_truth_offsets = [x for x in ground_truth_offsets if text_section_offset <= x <= text_sextion_end]
    ground_truth_offsets.sort()
    return ground_truth_offsets



def find_data_in_textsection(ground_truth_offsets , text_section_offset , text_section_len, offset_inst_dict):
    data_offsets = []
    for i in range(1, len(ground_truth_offsets)-1):
        distance = ground_truth_offsets[i+1] - ground_truth_offsets[i]

        inst_len = offset_inst_dict[ground_truth_offsets[i]].size 
        
        if distance!=inst_len:
            # print('offset_ranges[i]: ',ground_truth_offsets[i] , 'offset_ranges[i-1]: ',ground_truth_offsets[i-1], ' inst_len: ',inst_len  )
            
            print(ground_truth_offsets[i],' ' ,hex(ground_truth_offsets[i]) , offset_inst_dict[ground_truth_offsets[i]], ' len',offset_inst_dict[ground_truth_offsets[i]].size )
            print("\nByte GAP ###### ",distance ,' Missing bytes: ', distance - inst_len)
            
            for j in range( ground_truth_offsets[i] +inst_len , ground_truth_offsets[i+1]  ):
                data_offsets.append(j)
                if offset_inst_dict[j]:
                    print("# ",j, offset_inst_dict[j].mnemonic, offset_inst_dict[j].op_str , 'inst len:',offset_inst_dict[j].size )
                else:
                    print("# ",j, " invalid ")
            print('\n')
        else:
            print(ground_truth_offsets[i],' ', hex(ground_truth_offsets[i]) , offset_inst_dict[ground_truth_offsets[i]].mnemonic,offset_inst_dict[ground_truth_offsets[i]].op_str ,' len',offset_inst_dict[ground_truth_offsets[i]].size)
            pass
    return data_offsets
            

In [5]:





def linear_sweep(offset_inst , target_offset):
    inst_sequence = ''
    max_seq_length = 15

    
    current_offset = target_offset
    for q in range(max_seq_length):

        if current_offset in offset_inst: #if end of text section
            current_instruction = offset_inst[current_offset]
            if current_instruction is None:


                return  None
                
            current_offset = current_offset + current_instruction.size
            inst_sequence+= str( hex(current_instruction.address)) +" "+ current_instruction.mnemonic +' '+ current_instruction.op_str+ ' ; ' 

            if current_instruction.mnemonic in ["ret", "jmp"]: #break linear sweep
                return inst_sequence

    return inst_sequence


JSON_DATA= {}

for bin_file_path in bin_files:

    
    md = Cs(CS_ARCH_X86, CS_MODE_64)
    md.detail = True
    offset_inst = {}

    
    with open(bin_file_path, 'rb') as f:

        try:
            elffile = ELFFile(f)
           
            textSection = elffile.get_section_by_name('.text').data()
        
            text_section_offset = elffile.get_section_by_name('.text')['sh_offset']
          
            ground_truth_offsets = get_ground_truth_ghidra(bin_file_path, text_section_offset , len(textSection))
            
        except Exception as e:
            print("An error occurred:", e ,bin_file_path)
            continue

    for byte_index in range(len(textSection)):
        
    
        try:    

            instruction = next(md.disasm(textSection[byte_index: byte_index+15 ], text_section_offset + byte_index ), None)
            offset_inst[text_section_offset+byte_index] = instruction
            
            # if instruction:
            #     print("%d:\t%s\t%s _\t%x" %(int(instruction.address), instruction.mnemonic, instruction.op_str, instruction.size))
            # else:
            #     print("%d:\t%s " % (text_section_offset + byte_index  , 'invalid instruction') )
                
            

        except Exception as e:
            print(traceback.print_exc() )
            print(e)

    
    
    offset_inst_dict = collections.OrderedDict(sorted(offset_inst.items()))

    DATA_OFFSETS = find_data_in_textsection(ground_truth_offsets , text_section_offset , len(textSection) , offset_inst)

    print(DATA_OFFSETS)
    
    # for byte_offset in range(text_section_offset, text_section_offset+len(textSection)):
    #     inst_seq = linear_sweep(offset_inst_dict ,  byte_offset )
    #     print('\n','# # '*10)
    #     print(inst_seq)
    # print(ground_truth_offsets)
    # print(offset_inst.keys())
    print('\n','# # '*30)


4196   0x1064 push rbx  len 1
4197   0x1065 lea rbx, [rip + 0xf98]  len 7
4204   0x106c mov edx, 0x68  len 5
4209   0x1071 xor eax, eax  len 2
4211   0x1073 mov rsi, rbx  len 3
4214   0x1076 mov edi, 2  len 5
4219   0x107b call 0x1050  len 5
4224   0x1080 mov rsi, rbx  len 3
4227   0x1083 mov edx, 0x78  len 5
4232   0x1088 xor eax, eax  len 2
4234   0x108a mov edi, 2  len 5
4239   0x108f call 0x1050  len 5
4244   0x1094 xor eax, eax  len 2
4246   0x1096 pop rbx  len 1
4247   0x1097 <CsInsn 0x1097 [c3]: ret >  len 1

Byte GAP ######  9  Missing bytes:  8
#  4248 nop dword ptr [rax + rax] inst len: 8
#  4249  invalid 
#  4250 test byte ptr [rax], al inst len: 2
#  4251 add byte ptr [rax], al inst len: 2
#  4252 add byte ptr [rax], al inst len: 2
#  4253 add byte ptr [rax], al inst len: 2
#  4254 add byte ptr [rax], al inst len: 2
#  4255 add bl, dh inst len: 2


4256   0x10a0 endbr64   len 4
4260   0x10a4 xor ebp, ebp  len 2
4262   0x10a6 mov r9, rdx  len 3
4265   0x10a9 pop rsi  len 1
4

In [6]:
print('test')

test


SyntaxError: unmatched ']' (2815146837.py, line 1)